# Pipeline 2: Retrieval of patient information of patients that are still alive

This notebook presents all code used for the evaluation of the code produced for the second pipeline, as well as the related relational algebra

# Relational algebra

$D_{patients}(subject\_id, gender, anchor\_age, anchor\_year, anchor\_year\_group, dod)$

$patients\_nd = \sigma_{dod = Null}(patients)$


$patients\_nd\_by = \pi_{*, birth\_year(*)}(patients\_nd)$

$patients\_nd\_by = \rho_{patient\_id/subject\_id, age\_at\_care/anchor\_age, year\_of\_care/anchor_year}(patients\_nd\_by)$

$final\_data = \pi_{patient\_id, gender, age\_at\_care, year\_of\_care, anchor\_year\_group, birth\_year}(patients\_nd\_by)$

In [ ]:
# Necessary imports and setup of connection to the database

import polars as pl
from sqlalchemy import create_engine, inspect, Table, MetaData, Column, Integer, String, DateTime, Float, Boolean
import numpy as np

engine = create_engine(r"sqlite://path to where mimic4.db is stored") #change this to the path where mimic.db is stored

In [ ]:
# Ground truth code, used for comparison

metadata = MetaData()

patients = Table(
   'patients', metadata, 
    Column('subject_id', Integer, primary_key=True),
    Column('gender', String, nullable=False),   
    Column('anchor_age', Integer, primary_key=True),
    Column('anchor_year', Integer, nullable=False),
    Column('anchor_year_group', String, nullable=False),
    Column('dod', DateTime, nullable=True)
)
metadata.reflect(bind=engine)

query = """
SELECT *
FROM patients
WHERE dod IS NULL

"""
df = pl.read_database(query=query, connection=engine.connect())
df2 = df.with_columns((pl.col("anchor_year") - pl.col("anchor_age")).alias("birth_year"))

df2 = df2.rename({"subject_id": "patient_id", "anchor_age": "age_at_care", "anchor_year": "year_of_care"})

final_data_gt = df2.select([
    "patient_id", "gender", "age_at_care", "year_of_care", 'anchor_year_group', "birth_year"])

final_data_gt

# Code Valdity 

In the code block below, the generated code produced by the LLM can be pasted and executed, to assess if the code works without any runtime errors.

In [ ]:
# Paste generated code here 

# Code Correctness

To assess the code correctness, the code below can be executed. If it is correct, True should be returned. Otherwise, manual inspection in the code block below should be executed. It is assumed the presented solution is stored in a variable called 'final data'

In [ ]:
# final_data = df
final_data_gt.equals(final_data)

In [ ]:
#  Execute this code if the dataframes are not equal to see the differences
df_combined_pandas = df_combined.to_pandas()
final_data_pandas = final_data.to_pandas()

df_diff = df_combined_pandas.compare(final_data_pandas)

if not df_combined_pandas.equals(final_data_pandas):
    print(df_diff)

# Relative Efficiency

To record the average running time for the generated solution, the generated solution can be added to the code block below and executed. The average executrion time should be recorded per solution, for later determination of the relative efficiency

In [ ]:
import time
import numpy as np
import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')

execution_times = []

for i in range(10):
    start_time = time.time()

    # Paste generated code here

    end_time = time.time()

    execution_time = end_time - start_time
    execution_times.append(execution_time)


average_execution_time = np.mean(execution_times)

print(f"The code executed in average {average_execution_time} seconds over 10 runs.")

In [ ]:
# To calculate the relative efficiency of the generated solutions
def relative_efficiencies(times):
    tmin = min(times)
    tmax = max(times)
    efficiencies = [100 * (1 - (tc - tmin) / (tmax - tmin)) for tc in times]
    return efficiencies

time_gt = [48.2] # ground truth time
solutions = [] # Add the times from the 10 runs here
times = time_gt + solutions

re = relative_efficiencies(times)
# Calculate average
average = np.mean(re[1:])
print(f'Average: {average}')
